# RedisDB 
In this notebook, we will demonstrate how to use RedisDB, for accessing and querying data efficiently. RedisDB is designed to work seamlessly with modern analytical workloads, making it a powerful tool for data analysis, research, and question-answering systems.

To begin, ensure you have `redis` installed in your Python environment. You can easily install it using `pip install redis`.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/master/Demo/redis.ipynb)

In [ ]:
!pip install indox
!pip install redis
!pip install semantic_text_splitter
!pip install sentence-transformers

## Setting Up the Python Environment

If you are running this project in your local IDE, please create a Python environment to ensure all dependencies are correctly managed. You can follow the steps below to set up a virtual environment named `indox`:

### Windows

1. **Create the virtual environment:**
```bash
python -m venv indox
```
2. **Activate the virtual environment:**
```bash
indox_judge\Scripts\activate
```

### macOS/Linux

1. **Create the virtual environment:**
   ```bash
   python3 -m venv indox
```

2. **Activate the virtual environment:**
    ```bash
   source indox/bin/activate
```
### Install Dependencies

Once the virtual environment is activated, install the required dependencies by running:

```bash
pip install -r requirements.txt
```


### Load Hugging face API key 


In [1]:
import os
from dotenv import load_dotenv

load_dotenv('api.env')

HUGGINGFACE_API_KEY = os.environ['HUGGINGFACE_API_KEY']

In [2]:
from indox import IndoxRetrievalAugmentation
indox = IndoxRetrievalAugmentation()

INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            


Initialize a language model and an embedding model using the indox library with Hugging Face . The HuggingFaceModel class is used to create an instance of the Mistral-7B-Instruct model for tasks like question answering

In [3]:
from indox.llms import HuggingFaceModel
from indox.embeddings import HuggingFaceEmbedding
mistral_qa = HuggingFaceModel(api_key=HUGGINGFACE_API_KEY,model="mistralai/Mistral-7B-Instruct-v0.2")
embed = HuggingFaceEmbedding(api_key=HUGGINGFACE_API_KEY,model="multi-qa-mpnet-base-cos-v1")

INFO: Initializing HuggingFaceModel with model: mistralai/Mistral-7B-Instruct-v0.2
INFO: HuggingFaceModel initialized successfully
INFO: Initialized HuggingFaceEmbedding with model: multi-qa-mpnet-base-cos-v1


### Load Sample Data

In [4]:
file_path = "sample.txt"
with open(file_path, "r") as file:
    text = file.read()

use the `RecursiveCharacterTextSplitter` class from the indox library to divide a large text into smaller, manageable chunks

In [5]:
from indox.splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(400,20)
content_chunks = splitter.split_text(text)




### Set up vector store


In [6]:
from indox.vector_stores import RedisDB
redis_store = RedisDB(
    host='host',
    port=port,
    password='password',
    embedding=embed
)


### Storing Data in the Vector Store

In [7]:
redis_store.add(texts=content_chunks)


INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


Batches: 100%|██████████| 2/2 [00:00<00:00,  6.99it/s]


INFO: Added 38 texts successfully to the vector store.


['a34fba11-915e-4290-ab21-616ac23d0c28',
 'bd06dc46-375d-4c35-a92b-bf2de9909b0a',
 'd0e00a14-6760-4f1b-aad0-7b6d306fbb24',
 '7ecd5729-b8b5-4161-ab26-b096394a7837',
 '58aa1b45-c47a-4a99-96a1-d704073f6caf',
 '47514120-de26-492f-9f8c-747ec15bdbff',
 '3a1700c0-c6b6-4cd2-b57f-a31f30ddf1e7',
 'e59cc5f1-fe07-4d0c-8903-2b4ec6566489',
 '46dd468a-f2d0-41c6-97aa-b8cc2466b386',
 '8e6fcb25-0c55-4e6b-8495-06341e3e0fb6',
 '775fa1fd-94a1-44b3-aa5c-79d7a01f3d90',
 'c1857a44-72ac-4590-89da-a86d3e31dd6e',
 '85b4a7b0-ff15-47fd-b55b-a7d5f42188f2',
 'fe4612dd-4208-4158-9e44-fd3c0e1b057a',
 '52e736cb-64b8-4ad4-8ee0-447543694230',
 'be2f70c1-0d28-489c-91dd-d0439db932db',
 '33aca7c9-f8e5-4978-9f03-70fbfa59d160',
 'a12ac9b8-47f6-40d4-8202-3588b9c57500',
 '11c4eca6-7604-4844-bab9-ffd2103e5acb',
 '6f218133-cea2-4a76-8c41-f19c6a33ddd6',
 '6c3027f1-9136-4e0b-b10e-2312a23ee698',
 'f960bb89-5272-41d9-9d33-3329d051c4e6',
 'cc74141b-eb66-4cb7-99af-9ed38ea455b6',
 'd06d10fc-f68e-4da5-bcb0-a5a00e7140b2',
 '2974aeda-edee-

In [8]:
retriever = indox.QuestionAnswer(vector_database=redis_store,llm=mistral_qa,top_k=5)


In [9]:
query = "How cinderella reach her happy ending?"
answer = retriever.invoke(query=query)


INFO: Retrieving context and scores from the vector database
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


Batches: 100%|██████████| 1/1 [00:00<00:00, 110.18it/s]


INFO: Generating answer without document relevancy filter
INFO: Answering question
INFO: Sending request to Hugging Face API
INFO: Received successful response from Hugging Face API
INFO: Query answered successfully


In [10]:
answer

'Cinderella reached her happy ending when she made a wish and the bird granted it. The wish came true when the king threw a grand festival for his son to choose a bride, and Cinderella attended in disguise using the magic slippers left by the bird. At the ball, the prince was instantly drawn to Cinderella, but she left before he could catch her identity. With the help of the slipper, the prince was able to trace Cinderella down, and they'